# Tech Salary Exploration 2020-2022

## Loading and Cleaning Dataset

**To get the data into a useful format, the categorical and ordinal features need to be identified and transformed.  It wouldn't make sense to compare experience as a text field or company size as a charater, so these fields will be converted to ordinal factors.  Other fields like job title and currency should be represented as unordered categorical features (i.e. factors).  
To help reduce the complexity of the dataset, another column should also be added that reduces the number of job titles into a smaller subset of categories by pulling only the primary type of the role. It may also help to pull out features of the title such as _'manager'_ or _'lead'_ or _'intern'_ if they are prevalent enought to make a difference.**

In [ ]:
library(tidyverse)


orig_data <- read_csv('../input/aiml-salaries/salaries.csv')  %>%
    mutate(across(c(job_title, salary_currency, employee_residence, company_location), as.factor),
           experience_level = factor(experience_level, levels = c('EN', 'MI', 'SE', 'EX'), ordered = T ),
           employment_type = factor(employment_type, levels = c('FL','PT','CT','FT')),
           company_size = factor(company_size, levels = c("S", "M", "L"), ordered = T)
          )

orig_data %>% 
    summary()

### Cleaning Job Titles
**The job titles vary significantly, and many of the same types of jobs are referenced in different ways.  To make the job title more useful, a new encoding field will store the `super` class of the job indicated.**

**There are also many job titles that include management or seniority measures.  These can be collected separately as the role level, with the default role level being "1".**

In [ ]:
plot_data <- orig_data %>% 
    group_by(job_title) %>%
    summarize(count = n()) %>%
    mutate(index = rank(count, ties.method='first')) %>%
    arrange(desc(count))

job_titles <- as.vector(plot_data$job_title)
plot_data %>%
    filter(count > 3) %>%
    ggplot() +
        geom_col(aes(x=-index, y=count, fill = count)) +
        scale_x_continuous(breaks = -63:-1, labels=job_titles) + 
        labs(x = 'Job', y = 'count') +
        theme_bw() +
        theme(axis.text.x = element_text(angle = 60, vjust = 0.5, hjust=0.5))

In [ ]:
# str_detect with ability to pass in set of patterns to check
str_detect_any <- function(txt, patterns){
    return( any(
        vapply(patterns, function(x, t) str_detect(t, x), t = txt, FUN.VALUE = T)
    ))
}

# Simplify job_title field
reduced_job_title <- function(title){
    title <- str_to_lower(title)
    if( str_detect_any(title, c("data scientist", "data science")) ){
        "Data Scientist"
    } else if( str_detect_any(title, c("data engineer", "data engineering"))){
        "Data Engineer"
    } else if( str_detect_any(title, c("software engineer", "software engineering", "develop"))){
        "Software Engineer"
    } else if( str_detect_any(title, c("data analyst", "data analytics"))){
        "Data Analyst"
    } else if( str_detect_any(title, c("machine learning engineer", "machine learning engineering", "ml", "nlp"))){
        "Machine Learning Engineer"
    } else if( str_detect_any(title, c("applied scientist", "applied science"))){
        "Applied Scientist"
    } else if( str_detect_any(title, "research") ){
        "Researcher"
    } else if( str_detect_any(title, c("engineer", "engineering"))){
        "Engineer"
    } else if( str_detect_any(title, c("analyst", "analytics"))){
        "Analyst"
    } else if( str_detect_any(title, c("scientist", "science"))){
        "Scientist"
    } else if( str_detect_any(title, "architect")){
        "Architect"
    } else {
        "Other"
    }
}
        
# Identify role seniority from job_title
seniority <- function(title){
    title <- str_to_lower(title)
    if( str_detect_any(title, c("manage", "head", "direct"))){
        "Manager"
    } else if( str_detect_any(title, "consult") ){
        "Consultant"
    } else if( str_detect_any(title, c("senior","lead", "principal", "sr"))){
        "Senior"
    } else if( str_detect(title, "\\d") ){
        str_extract(title, "\\d")
    } else if( str_detect_any(title, c("junior", "jr", "intern"))){
        "Junior"
    } else {
        "1"
    }
} 


data <- orig_data %>%     
        mutate(job_super_title = sapply(job_title, reduced_job_title) %>% 
                    factor(),
               job_seniority_title = sapply(job_title, seniority) %>% 
                    factor(levels = c("Junior", "Consultant", "0", "1", "2", "3", "Senior", "Manager"), ordered = T) )
    
data %>%
    select(job_super_title, job_seniority_title) %>%
    summary()

**Goal is to miss the fewest titles with this method - looks like only a few slip through (mostly managerial roles that will be identified by their seniority)**

In [ ]:
other_title <- data %>% filter(job_super_title == 'Other') %>% group_by(job_title) %>% count()
other_title

## Rise of Remote Work

**Since the beginning of the pandemic in 2020 remote work has grown increasingly popular.  This data can be used to understand the impact of remote work on employee salaries and locations.**

**Some questions that come immediately to mind are:**
* **What yearly trends exist in remote work offerings?**
* **Which roles are most amenable to remote work?**
* **Where do remote workers choose to live?**
    * **Which countries offer remote work?**
* **How much pay difference exists between remote workers?**
    * **Is there a difference related to the year the role was offered?**


### What yearly trends exist in remote work offerings?

**The first and simplest question is what are the yearly trends in remote work offers?  How much did remote work offers change over the 3 years.  This can be seen by plotting the frequency of on-site, hybrid, and remote roles over the 3 years given.**

In [ ]:
# Collect dataset for remote work
remote_work_data <- function(dataset, main_col, group_cols = NULL){
    main_col <- as.name(main_col)
    if( !is.null(group_cols) ){
        group_cols <- as.name(group_cols)
        ds <- dataset %>%
            group_by(!! main_col, !! group_cols)
    } else {
        ds <- dataset %>% 
            group_by(!! main_col)
    }
    ds <- ds %>%
        summarize(remote_ratio_avg = mean(remote_ratio), remote_ratio_sd = sd(remote_ratio), hybrid = (sum( remote_ratio > 0 & remote_ratio < 100 )), fully_remote = sum(remote_ratio == 100), no_remote = sum(remote_ratio == 0), count = n()) %>%
        mutate(full_remote_pct = fully_remote / count * 100, no_remote_pct = no_remote / count * 100) %>%
        mutate(hybrid_pct = hybrid / count * 100) %>%
        arrange(desc(count))

    return(ds)
}

remote_data <- data %>% 
    remote_work_data('work_year') 

remote_data %>%
    pivot_longer(c(full_remote_pct, hybrid_pct, no_remote_pct), names_to = "role_type", values_to = "percent") %>%
    ggplot(aes( fill = role_type)) +
        geom_col(aes( x = work_year, y = percent ), position = position_dodge()) +
        labs( title = "Type of Role Offered", subtitle = "by work year", y = 'Percent %', fill = "Role Type", alpha = NULL) +
        theme_bw()
        

In [ ]:
remote_data %>%
    ggplot() +
        geom_col(aes( x = work_year, y = remote_ratio_avg, fill = remote_ratio_sd)) +
        labs( title = "Average Ratio of Remote Work", subtitle = "by work year", y = 'Percent %', fill = "Remote Ratio SD", alpha = NULL) +
        theme_bw()

**The two charts above show how remote offerings in ML roles changes over the course of the pandemic.  Both have the obvious trend where remote offerings peak in 2021 (after employers had accepted the need for workers to stay home), but with in-office and hybrid offerings increasing in 2022.  The fall of hybrid work is most clear in the first chart, which shows how more roles offered a hybrid offering in 2021 than 2020, but these types of roles decreased dramatically in 2022.**

**The subtle color and width variation in the second chart represent the standard deviatio in the amount of remote time offered across all roles in a given year.  In 2021 there were both more remote roles and less variance in the remote time offered (because most roles were 50% or 100% remote).  In 2022 that variance increases noticeably as more roles are explicitly 0% or 100% remote.  2020 is somewhat in the middle, because the number of offerings of remote, hybrid, and on-site roles are more even.**

### Which roles are most amenable to remote work?

It should be easy to discern the roles that offer the most remote roles based on the way the data is split out.  Beyond just the job title, there could be differences in terms of the level of seniority and the year in which the job was offered that 

In [ ]:
# Size GGPlot figure 
fig <- function(width, heigth){
     options(repr.plot.width = width, repr.plot.height = heigth)
}

# Produce ggplot for remote work by main_col over grouping
remote_work_breakdown <- function(dataset, main_col, group_cols = NULL, width = 12, height = 8){
    ds <- remote_work_data(dataset, main_col, group_cols)
    main_col <- as.name(main_col)
    p <- ds %>%
        pivot_longer(c(full_remote_pct, hybrid_pct, no_remote_pct), names_to = "remote", values_to = "val") %>%
        ggplot(mapping = aes(), fig(width, height)) + 
            geom_col(aes(x = !! main_col, y = val, fill=remote), position=position_stack(0)) +
            geom_text(aes(x = !! main_col, y = 107, label=count, alpha = ifelse(remote=='full_remote_pct', 1, 0)), show.legend = F) +
            geom_text(aes(x = !! main_col, y = if_else(fully_remote == 0, 103, 97), label=paste0(round((fully_remote)/count * 100), '%'), alpha = ifelse(remote=='full_remote_pct', 1, 0)), show.legend = F) +
            geom_text(aes(x = !! main_col, y = if_else(hybrid == 0, no_remote / count * 100 + 3, (hybrid + no_remote) / count * 100 - 3), label=paste0(round((hybrid)/count * 100), '%'), alpha = ifelse(remote=='full_remote_pct', 1, 0)), show.legend = F) +
            geom_text(aes(x = !! main_col, y = (no_remote / count * 100 - 3), label=paste0(round((no_remote)/count * 100), '%'), alpha = ifelse(remote=='full_remote_pct', 1, 0)), show.legend = F) +
            labs( title = "Remote Work Offerings", subtitle = paste0("by ", main_col, if_else(is.null(group_cols), '', paste0(' and ', group_cols))), y = 'Percent %', fill = "Policy", alpha = NULL) +
            theme_bw() +
            theme(axis.text.x = element_text(angle = 60, vjust = 0.5, hjust=0.5))
    return(p)
}

remote_work_breakdown(data, 'job_super_title', NULL, 16, 8)


**For most roles, above 50% of offerings are fully remote.  From this chart it appears as though research roles are the least amenable to remote work, but the most likely to request hybrid work.  Surprisingly, roles in research, data science, data engineering, and (other) engineering all require onsite work about 30% of the time.  The seniority and year in which the role is offered may also impact the rate of remote work.**

In [ ]:
remote_work_breakdown(data, 'job_seniority_title')

**In terms of role level it seems that roles at all levels provide similar remote offerings.  For roles at level 3 there are only 2 samples (1 hybrid, 1 on-site), so this result can be discounted.  One thing to note is that the more senior roles offer a lot more hybrid and fully remote jobs - 88% of senior roles have a remote component, compared with only 72% for level 1 roles and 74% for management roles.** 

In [ ]:
remote_work_breakdown(data, 'job_super_title', 'work_year', 24, 8) +
    facet_grid(rows=vars(work_year)) 

In [ ]:
remote_work_breakdown(data, 'job_seniority_title', 'work_year', 24, 8) +
    facet_grid(rows=vars(work_year)) 

**In both the year-over-year charts above the same trend appears:**
* **In 2020 companies begin to offer lots of hybrid and remote opportunities**
* **In 2021 the trend continues with very few companies requiring workers to be in the office**
* **But in 2022 companies are beginning to favor on-site and hybrid offerings over fully remote, except perhaps when it applies to the most competitive senior roles.**

### Where do remote workers live?

**Remote workers usually have the opportunity to do their work from anywhere.  How many use this opportunity to get out of the big cities and tech centers?  Is there a significant compensation difference that depends on where workers are living?**

**First off though, how many workers live outside the country their employer is based in?**

In [ ]:
data %>%
    mutate(across(c(employee_residence, company_location), as.character)) %>%
    filter(remote_ratio == 100, employee_residence != company_location) %>%
    group_by(employee_residence, company_location) %>%
    count(name='count') %>%
    arrange(desc(count)) %>%
    ggplot() +
        geom_col(aes(x = company_location, y = count, fill = employee_residence)) + 
        facet_wrap(~employee_residence, scales = 'free_x') +
        labs(title="Where Remote Employees Live", subtitle="vs. Company Location",
             x = "Company Location", y = "Count", fill = "Employee\nResidence") +
        theme_bw()

**Of all the places remote workers are living, it looks like India, Brazil, France and Pakistan are the most common.  It doesn't come as a surprise that the most common company location among remote workers is the US.**

**The location data is very high level, so exploration into the specifics of where workers live is limited to their country of residence.**

### What is the difference in salary between remote and hybrid or on-site workers?

**Salaries may be different for the same role in different countries, but what about remote workers vs. workers expected to be in office at least part time?**

**Before looking at salary differences it is also important to convert all salaries to the same currency (based on most up-to-date exchange rates).**

In [ ]:
# Daily exchange rate for global currencies on the most recent date
exchange_rates <- read_csv('../input/currency-exchange-rates/exchange_rates.csv') %>%
    mutate(date = lubridate::dmy(date)) %>% 
    filter(date == max(date)) %>%
    select(currency, value) 

# USD to EUR exchange rate (EUR based)
USD_RATE = exchange_rates$value[exchange_rates$currency == 'USD']

# Convert exchange to USD base
exchange_rates <- exchange_rates %>% 
    mutate(USD_value = value / USD_RATE)

# Add USD salary by conversion
converted_data <- data %>%
    inner_join(exchange_rates, by = c("salary_currency"="currency")) %>%
    mutate(usd_salary = USD_value * salary)

summary(converted_data %>% select(usd_salary, salary, USD_value))

In [ ]:
converted_data %>%
    filter(usd_salary < 1*10**6) %>%
    mutate(remote = remote_ratio == 100) %>%
    group_by(remote, work_year, job_super_title) %>%
    summarize(count = n(), salary_avg = mean(usd_salary), salary_med = median(usd_salary)) %>%
    ggplot() +
        geom_col(aes(x = work_year, y = salary_avg, fill=remote), position='dodge') +
        labs(title = 'Yearly Average Salary', subtitle = 'by Job Title and Remote Status', x = 'Year', y = 'Salary (USD)', fill = 'Remote') +
        facet_wrap(~job_super_title) + 
        theme_bw()

In [ ]:
converted_data_chart <- converted_data %>%
    # remove skew from outlier - makes chart hard to read
    filter(usd_salary < 1*10**6) %>%
    mutate(across(c(employee_residence, company_location), as.character)) %>%
    group_by(employee_residence) %>%
    summarize(count = n(), salary_avg = mean(usd_salary), salary_med = median(usd_salary)) %>%
    # only common residences to make chart easier to read
    filter(count > 5) %>%
    ungroup() %>%
    mutate(index = -rank(salary_avg, ties.method = 'first')) %>%
    arrange(desc(index))

labels <- converted_data_chart$employee_residence
converted_data_chart %>%
    pivot_longer(c(salary_avg, salary_med), names_to = 'stat', values_to = 'val') %>%
    ggplot() + 
        geom_col(aes(x = index, y = val, fill = stat), position = position_dodge()) +
        scale_x_continuous(breaks = -1:-length(labels), labels=labels) +
        labs(title="Overall Salary by Country of Residence", subtitle="Average and Median", x = "Country of Residence", y = "Salary (USD)", fill = "Stat") +
        theme_bw()

**This chart uses both mean and median salary to capture the influence of outliers, which tend to skew the mean much higher than the median.  Overall though, among countries with high representation (more than 5 employee residents), there isn't a massive difference between mean and median salaries.  The best salaries offered (in terms of USD) seem to be in Brazil, while the lowest pay in USD is in Pakistan.  These values don't account for job titles or seniority, but it still offers some insight into the wide differences in compensation in terms of USD by country.**